In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import random
import seaborn as sns

from sklearn import metrics

# import the KNNimputer class
from sklearn.impute import KNNImputer

In [2]:
def couleur_léatoire_hex() :
    """
    Générateur de couleur aléatoire dans le format hexadecimal

    paramètres :  
    ------------
        Aucun paramètre à fournir

    Returns : 
    ---------
        String : Code hexadecimal de la couleur fabriquée sous forme d'une variable chaine de caractères.
    """
    s = "#"
    a = np.random.choice(["0","1","2","3","4","5","6","7","8","9","A","B","C","D","E","F"] , 6 ,replace=False)
    for i in a : s += i
    return s

def camemberg ( labels = ["Brice","Romeo","Marthe","Franck","Arnaud"] , sizes=[28,18,23,30,15] ,titre = "CAMENBERG", figure_size =(8,8) , rot = 30 ) :
    """
    La fonction permet d'afficher un camemberg selon les entrée fournies
    
    Paramètre :
    -----------
        labels : Array_like 
                Labels des données à afficher sous forme de camemberg
        sizes :  Array_like 
                Grandeurs quantitatives correspondants aux labels fournis
        titre : str
                Titre du graphique
        figure_size : tuple
                respectivement la largeur et la hauteur du graphique sous forme de tuple
        
    Return :
    --------
        None
    """
    # Data to plot
    label = np.array(labels)
    size = np.array(sizes)
    colors = [couleur_léatoire_hex() for i in range(size.size) ]
    explode = np.array([ 0. for i in range(size.size) ])  # explode 1st slice
    explode[size == size.max()] = 0.08

    # Plot
    plt.figure( figsize = figure_size )
    plt.title(titre, color ='brown' , size = 18 )
    plt.pie(size, explode=explode, labels=label, colors=colors, autopct='%1.1f%%', shadow=True, startangle=rot, textprops={'fontsize': figure_size[0]*2})
    plt.axis('equal')
    plt.show()


def nan_proportion ( data , figsize=( 7 , 7 ) , titre = "\nValeurs manquantes et infinies dans les données brutes\n" , rot = 15 ):
    """
    La fonction trace un Camemberg donnant les proportions de valeurs manquantes infinies et finies adans les données
    
    Paramètres :
    ------------
        data : pandas.core.frame.DataFrame
            Données à utilisée par la fonction sous forme d'un DataFrame de la bibliothès pandas                
        figsize : tuple
            respectivement la largeur et a hauteur du camemberg à représenter
            
    return :
    --------
            None
    """
    a =  data.isna().mean().mean()
    b = data.isin([np.inf,-np.inf]).mean().mean()
    camemberg(sizes = [ a , b , 1-a-b ] , labels = ["NAN","Infinie","Valeurs Finies"] ,titre = titre,figure_size=figsize , rot = rot)
    
def plot_nan_proportion( data ,  plt_type = "lns", n_bar = 30, figsize = (7,7)) :
    """
    Traces l'Histogramme de la distribution des valeurs manquantes par colonne ou par ligne des données fournies à la fonction
    
    Paramètres :
    ------------
        data :  pandas.core.frame.DataFrame
            Données à utilisée par la fonction sous forme d'un DataFrame de la bibliothès pandas
        plt_type :  str
            type de répresentation à réaliser. peut prendre les valeurs "lns" et "col" respectivement pour une représentation des valeurs manquantes par lignes("lns") ou par colonnes("col")
            La valeur par defaut est "lns"
        n_bar : int
            Nonbre de barres dans l'histogramme à afficher. Corresponds au paramètre "bins" de la methode plt.hist()            
        figsize : tuple
            respectivement la largeur et a hauteur du camemberg à représenter
            
    Return :
    --------
        None
    """
    plt.figure()
    if plt_type =="col" :
        sns.displot( data.isna().mean()*100, kde=True,color=couleur_léatoire_hex() , bins=n_bar, height=figsize[1], aspect=figsize[0]/figsize[1] )
        plt.xlabel('valeurs manquantes (% )', size= 15)
        plt.ylabel("Nombre de variables", size =15)
        plt.title("Proportions de valeurs manquantes sur les colonnes",size=25)
        plt.show()
    elif plt_type == "lns": 
        sns.displot( data.isna().mean(axis=1)*100, kde=True,color=couleur_léatoire_hex(), bins=n_bar, height=figsize[1], aspect=figsize[0]/figsize[1] )
        plt.xlabel('valeurs manquantes (% )', size= 15)
        plt.ylabel("Nombre de Lignes", size =15)
        plt.title("Proportions de valeurs manquantes sur les lignes",size=25)
        plt.show()
        
def tableau_valeur_manquante(data):
        """
        La fonction renvoie un DataFrame panda qui fournie le nombre et le pourcentage des valeurs manquantes et infinies pour chaque colonnes
        Les variables sont ensuite ordonnées selon les valeurs décroissantes du nombre de valeurs manquantes trouvées
        
        Paramètres :
            None
        ------------
            data : pandas.core.frame.DataFrame
                Dataframe à utiliser dans la fonction pour le calcul des valeurs manquantes et infinies
        """
        # Construction du tableau de valeurs manquantes et leur pourcentage pour chaque colonne et Réattribution des noms aux colonnes du tableau
        mis_val_table = pd.concat([ data.isin([np.inf,-np.inf]).sum() , data.isnull().sum() , 100*data.isin([np.inf,-np.inf]).mean() , 100*data.isnull().mean()  ] , axis=1)
        mis_val_table = mis_val_table.rename( columns = { 0 : "Nb de valeurs infinies" , 1 : "Nb de Valeurs manquantes", 2 : "Inf Proportions ( % )", 3 : "Nan Proportions ( % )" } )
        
        # Sort the table by percentage of missing descending
        mis_val_table = mis_val_table.sort_values( by = "Nan Proportions ( % )" ,axis =0 ,ascending=False).round(2)
        
        return mis_val_table 

def print_features_importances( model, X , n = 20 ,figsize = (20,15), get_importance = False ,error_raise = False) :
    """
    La fonction permet d'afficher l'importance globale de variables pour un modèle donnée de machine leaning 
    
    paramètres :
    ------------
        model : machine learning model 
            modèle a utiliser pour la recherche de l'importante de variables/features
        X : pandas.core.frame.DataFrame
            données à fournir au modèle
        n : int 
            nombre de variables à afficher la valeur par defaut est de 10
        figsize : tuple
            respectivement la largeur et la hauteur du graphique qui sera affiché. la valeur par defaut est (20,15)
        get_importance : bool
            Dire si la fonction doit ou non renvoyer la liste des features importances. La valeur par defaut est False   
        error_raise : bool
            Dire si le programme doit continuer de tourner ou interrompre son excussion quand elle rencontre une erreure.
            La valeur par defaut est False
    
    return :
    --------
        importance_features : Array_like
    
    Raises:
    -------
        AttributeError : Une exception est eventuellement levée lorsque lorsque le modèle fourni ne possède pas d'attribut 'features_importances_'
    """
    # Acquisition de l'importance des fetaures
    try :
        importance = model.feature_importances_
    except AttributeError:
        if error_raise : 
            raise AttributeError("Le modèle que vous avez fourni ne possède pas l'attribut 'features_importances_'")
        else :
            print("Le modèle que vous avez fourni ne possède pas l'attribut 'features_importances_'")
        return
    
    
    importance /= max(importance)
    # création du dataframe
    importance_features = pd.DataFrame({ "Features" : X.columns ,"Importance": importance }) 
    # Triage par ordre d'importance
    importance_features.sort_values(by="Importance", inplace=True, ascending=True)                     

    plt.figure(figsize= figsize)
    plt.barh(importance_features.iloc[-n:,0],importance_features.iloc[-n:,1], color = [couleur_léatoire_hex() for i in range(n)])
    plt.title("\nImportance des différentes variables", size = 2*figsize[0])
    plt.xlabel("Importance" , size = 1.5*figsize[0])
    plt.show()
    if get_importance : 
        return importance_features     

def concentration_gini ( data) :
    """
    Mesure de concentration des données selon le modèle de Gini
    
    Paramètres :
    ------------
        data : pandas.core.frame.DataFrame
            données a utiliser par la fonction
            
    Return :
    --------
        gini : float
    
    """
    data = data[~data.isna().values]
    data =  np.append(0, np.cumsum( np.sort(data) )/data.sum())
    gini = 2*( 0.5 - ( data.sum() - 0.5*data[-1] - 0.5*data[0] )/data.size  )
    return gini

def concentration_brice ( data ):
    """
    Mesure de concentration de données selon le modèle défini de Brice KENGNI ZANGUIM
    
    Paramètres :
    ------------
        data : pandas.core.frame.DataFrame
            données a utiliser par la fonction
            
    Return :   float
    -------- 
    """
    q_a , q_b, q_c = 0.8 , 0.5 , 0.2 # Quantilles extrêmes pris en compte
    data = data[~data.isna().values]
    box_over_domain = np.exp( -(q_a-q_b)*(q_a-q_c)*(q_b-q_c)  + \
                              ( np.quantile(data,q_a) - np.quantile(data,q_b) ) *  \
                              ( np.quantile(data,q_a) - np.quantile(data,q_c) ) *  \
                              ( np.quantile(data,q_b) - np.quantile(data,q_c) ) /  \
                              ( data.max() - data.min() )**3
                             )
    peer_to_peer_distance_mean =  1 #( data.max() - data.min()  )/data.size
    distance_to_min_mean =  (data - data.min()).var()/(( data.max() - data.min() )**2/12. ) 
    return np.sqrt( box_over_domain*peer_to_peer_distance_mean*distance_to_min_mean  )

def concentration ( data , mode = 'brice' , around = 5 ):
    out = {}
    df = pd.concat([data.select_dtypes(include=[float]), data.select_dtypes(include=[int])], axis=1)
    for i in df.columns :
        a =  data[i]
        if mode =='brice' :
            out[i] = round(concentration_brice(a) , around)
        elif mode =='gini' :
            out[i] = round(concentration_gini (a),around)
        elif mode == 'both':
            out[i] = round(concentration_brice(a)*concentration_gini (a)  , around)
    return out

def random_replace ( data , mask,std = 0.5) :
    """Generateur d'une liste de nombre aléatoires repartis dans un domaine entre la mediane et la moyenne des données"""
    return abs( np.random.normal( 0.5*(data.median( ) + data.mean( ) ),\
                                   ( std*(data.quantile( 0.8 ) - data.quantile( 0.2 )) ) ,\
                                   ( mask.sum() ) \
                                   ) )

def unique_boxplot(data , label ) :
    data = data[~data.isna().values]
    print(f"\n\t\tSize : {data.shape} - Mean : {round(data.mean(),2)} - Med : {round(data.median(),2)} -  IQR : {round(data.quantile(0.75) - data.quantile(0.25)  ,2)} -  Concentration : {round(concentration_brice(data),4)}")
    plt.figure(figsize=(18,2))
    plt.boxplot(data, labels = label, vert =False, whis=1.5, showmeans =True, widths=0.4)
    plt.ylabel('')
    plt.title(f'Boxplot : {label[0]}' , size=15)
    plt.show()

def all_boxplots(data , type_var = "float" , limit = 0.1 , kind = 'min'):

    if type_var =="float" :
        df = data.select_dtypes(include=[float])
    elif type_var =="int" :
        df = data.select_dtypes(include=[int])
    else :
        df = data.select_dtypes(include=[float, int])
        #df = pd.concat([data.select_dtypes(include=[float]), data.select_dtypes(include=[int])], axis=1)
    if kind == 'max' :   # On affiche les boxplot de toutes les colonnes sans exception
        for col in df.columns :
            unique_boxplot( df[col] , [col] )
    elif kind =='min' :  # On affiche uniquement les boxplot de colonnes avec une concentration inférieure à la limite établie
        conc = concentration(df) 
        colons , mesure = list(conc.keys())  , list(conc.values())
        mesure , colons = zip(*sorted(zip(mesure,colons) ,reverse=True))
        mesure , colons = np.array(mesure) , np.array(colons)
        if np.array(limit).size == 1 :
            colons = colons[mesure <= limit]
            for col in colons :
                    unique_boxplot( df[col] , [col] )
        elif np.array(limit).size == 2 :
            colons = colons[(limit[0] <=mesure) & (mesure <= limit[1])]
            for col in colons :
                    unique_boxplot( df[col] , [col] )

def plot_multiple_bar(data, group, observation, figsize = (9,6), show_count = True, normalize = True) :
    group_x = np.array(data[group].value_counts().index) #; group_x = group_x[~np.isnan(group_x)]  # Différents groupes d'individus dans lesquels on regroupe les donnéesGroup d'individus 
    group_y = data[observation].value_counts().index  #; group_y = group_y[~np.isnan(group_y)]   #  Modalité qu'on observe pour chaque groupe d'individus
    if normalize :                        # Nombre/Frequence de modalités pour chaque groupe d'individus
        count = { j : [round(( (data[group]==i)&(data[observation]==j) ).mean(),6)   for i in group_x  ] for j in group_y  } 
    else :
        count = { j : [( (data[group]==i)&(data[observation]==j) ).sum()   for i in group_x  ] for j in group_y  }
    
    x_axis = np.arange(group_x.size)
    width = 1./(group_y.size+2)
    plt.figure(figsize=figsize)
    for k in range( group_y.size ):
        pl = plt.bar( x_axis+width*(k-0.5*(group_y.size-1)), count[group_y[k]] ,width = width, label = group_y[k])
        if show_count : 
            for bar in pl: plt.annotate(bar.get_height() , xy=(bar.get_x()+0.1*width , 1.01*bar.get_height()) , fontsize=12)
    
    plt.xticks(x_axis ,group_x)
    plt.xlabel(group, size= 15)
    plt.ylabel("Quantité", size=15)
    plt.legend(loc='best')
    plt.show()
    
def density_compare(data , observation  , group , x_bornes = None) :
    plt.figure(figsize=(10,7))
    plt.title(f"Distribution de Densité de {observation} par {group}")
    for i in data[group].unique() :
        if i !="XNA":
            sns.kdeplot(data.loc[ data[group]== i  ,observation],label = f"{group} = {i}" , lw= 4  )
            #sns.displot( data.loc[ data[group]== i  ,observation], kde=True,color='g', bins=120, height=7, aspect=1.7 ,label = i)
    if x_bornes : plt.xlim(x_bornes[0] , x_bornes[1])
    plt.legend(loc = 'best')
    plt.ylabel("Distribution de densitié" , size = 15)
    plt.xlabel( observation , size=15)
    plt.show()

def get_train_test_dataframe ( data , seed = None , test_prop = 0.2, nan_in_test = False ) :
    """
    La fonction réalise un split des données fournies en données de test et d'entraineemnt
    
    Paramètres :
    ------------
        data : pandas.core.frame.DataFrame
            Donnée à utiliser dans l'execution de la fonction
        seed : int
        
        test_prop : float
            correspond à la proportion que doit avoir les données de test. 
            La valeur par defaut est de 0.2 et doit toujorus être comprise entre 0 et 1
    
    Return :  pandas.core.frame.DataFrame
    --------
        data_train 
        data_test
    
    """
    if nan_in_test :
        test_mask = data.sample(frac = test_prop , random_state= seed ).index
    else :
        test_mask = data[data.isna().sum(axis=1) == 0].sample(n = int(test_prop*data.shape[0])).index
    
    return data.drop(test_mask, axis=0) , data.loc[test_mask,:]

def print_confusion_matrix(confusion_matrix, class_names, figsize = (8,7), cmap = None): 
    import itertools
    
    """
    Affiche la transposée de la  matrice de confusion tel que fournie par 
    la fonction sklearn.metrics.confusion_matrix, sous forme de heatmap
    
    Paramètres :
    ------------
        confusion_matrix : Array_type
            matrice de confusion
        class_name : list or Array_type
            label à attribuer aux observations : 0 - 1 , oui - non , bon - mauvais , ok - pas ok
    
    Return : None
    --------
    
    """    
    confusion_matrix = confusion_matrix.T
    with plt.style.context(('ggplot', 'seaborn')):
        fig = plt.figure(figsize=figsize, num=1)
        plt.imshow(confusion_matrix, interpolation='nearest',cmap= plt.cm.Blues )
        plt.xticks([0,1],class_names , )
        plt.yticks([0,1],class_names)
        plt.xlabel('Vraie étiquette' , size = 3*np.min(figsize) )
        plt.ylabel('Etiqeutte prédite' , size = 3*np.min(figsize))
        for i, j in itertools.product(range(confusion_matrix.shape[0]), range(confusion_matrix.shape[1])):
                    plt.text(j, i,confusion_matrix[i, j], horizontalalignment="center",color="red" , fontsize = 2.2*np.min(figsize))
        plt.grid(None)
        plt.title('Confusion Matrix' , fontsize = 4*np.min(figsize))
        plt.colorbar() 
    
def prediction_function_threshold ( model=None , X=None, Y_proba=None , seuil = 0.5 ) :
    """
    La fonction permet d'évaluer la prediction d'un modèle donnée en fonction du modèle, de l'entrée, d'une probabilité fournie et d'un seuil 
    
    Paramètres :
    ------------
        model : modèle de machine learning à utiliser.
        X : pandas.core.frame.DataFrame
            donnée à fournir au modèle pour la prédiction. Si model est fourni alors X doit aussi être fourni et dans ce cas Y_proba n'est pas utilisé.
        Y_proba : Array_type
            probabilité qu'un individu soit du label positif. Si Y_proba est fourni alors model et X ne sont pas utilisés
        seuil : float
            seuil de probalité à utiliser pour la calcul de la prediction. la valeur par defaus est 0.5 et doit toujours être comprise entre 0 et 1
    
    Return : Array_type
        prediction
        
    """
    if ( type(model) == type(None) ) and ( type(X) == type(None) ): 
        return np.array( Y_proba > seuil , dtype = int)
    else :
        try : 
            return np.array( model.predict_proba(X)[:,1] > seuil , dtype = int)
        except :
            print("Le modèle que vous avez fourni ne possède pas de méthode 'predict_proba()'")
            return
        
#  Definition de la meilleure métrique 
#  Definition de la meilleure métrique 
def my_cost( y , y_pred , poids = 4  , seuil = np.linspace(0.08,0.9,100 ) , scorer = False ) : 
    """
    La fonction permet d'évaluer le cout métier d'un modèle donné 
    
    Paramètres :
    ------------
        y : Array_like 
            vraie valeur labels pour chaque individus ou observations
        y_pred : Array_like 
            vecteur probabilité ( d'être positif  ) peu aussi être le vecteur prediction du modèle pour chaque individus ou observations
            
    return : dict
    ------------
        out : dict 
        La dictionnaire renvoyé est de la forme : 
        { "cout" : Array_type de variation du coût en fonction du seuil , "cout_min" : valeur du coût minimal , "seuil_min" : seuil correspondant au cout minimal }
    """
    if type(y) == type(pd.DataFrame()) :
        label = pd.DataFrame( {"Y_test" : y.values.reshape( (y.shape[0],) ) , "Y_prob" : y_pred } )
    else :
        label = pd.DataFrame( {"Y_test" : y , "Y_prob" : y_pred } )

    out = {"cout" : []}
    if type(seuil) == type(0.2) : seuil = np.array([seuil])
    for s in seuil : 
        label["Y_pred"] = label["Y_prob"].apply( lambda x : int(x > s))
        label.loc[ (label["Y_test"] ==0) & (label["Y_pred"] == 0) , "decision"]  = "VN"
        label.loc[ (label["Y_test"] ==1) & (label["Y_pred"] == 1) , "decision"]  = "VP"
        label.loc[ (label["Y_test"] ==0) & (label["Y_pred"] == 1) , "decision"]  = "FP"
        label.loc[ (label["Y_test"] ==1) & (label["Y_pred"] == 0) , "decision"]  = "FN"
        out["cout"].append( ( label["decision"] == "FP" ).mean() + poids*( label["decision"] == "FN" ).mean() )

    out["cout"] = np.array(out["cout"])
    if scorer : 
        return out["cout"].min()
    else : 
        out["cout_min"] = out["cout"].min()
        out["seuil_min"] = seuil[out["cout"] == out["cout_min"]]
        out["cout"] = list(out["cout"])
        return out
        
def print_scores(model = None , X_test=None , Y_true=None , Y_proba = None  , line_width = 6 , seuil = np.linspace( 0 , 0.9 , 90 ) , plot_kind = "apr", give_results = False , show_graph = True) :
    scores = {} 
    beta = np.linspace( 0.7 , 2. , 2 )
    if ( type(model)!=type(None) ) and ( type(X_test)!=type(None) ) and ( type(Y_proba) == type(None) ) :
        try :
            Y_proba = model.predict_proba(X_test)[:,1]
        except :
            print("Le modèle que vous avez fourni ne possède pas de méthode 'predict_proba'")
            return
    
    ## Pour chaque coefficient beta de F_beta score, je vais calculter la variation du F_beta score avec le seuil
    if "b" in plot_kind :
        for b in beta :
            scores[f"beta = {b}"] = [  metrics.fbeta_score(Y_true , prediction_function_threshold(Y_proba = Y_proba ,seuil = s) , beta = b ) for s in seuil ]
    if "a" in plot_kind :
        scores[f"Accuracy"] = [ metrics.accuracy_score(Y_true , prediction_function_threshold(Y_proba = Y_proba ,seuil = s)  )  for s in seuil  ]      #  Accuracy en fonction du seuil
    if "r" in plot_kind :
        scores[f"Recall"] = [ metrics.recall_score(Y_true , prediction_function_threshold(Y_proba = Y_proba ,seuil = s)  ) for s in seuil  ]           #  Recall en fonction du seuil
    if "p" in plot_kind :
        scores[f"Precision"] = [ metrics.precision_score(Y_true , prediction_function_threshold(Y_proba = Y_proba ,seuil = s)  ) for s in seuil  ]     #  Precision en fonction du seuil
    if "h" in plot_kind :
        scores[f"Hamming Loss"] = [ metrics.hamming_loss(Y_true , prediction_function_threshold(Y_proba = Y_proba ,seuil = s)  ) for s in seuil  ]     #  Precision en fonction du seuil
    if "c" in plot_kind :
        scores["Fonction Coût"] = my_cost(Y_true , Y_proba , seuil = seuil ,scorer=False)["cout"]     #  Precision en fonction du seuil
    
    if show_graph :
        #  Affichage de la figure
        plt.figure(figsize=(16,11))
        plt.title("\nF_beta, accuracy, Précision et Recall VS seuil" , size=25)
        plt.xlabel(" Seuil de probabilité" , size= 18)
        plt.ylabel("SCORE" , size = 18)
        
        for label , y  in scores.items()  :
            plt.plot(seuil , y , lw = line_width ,ls = np.random.choice(["dashed","dotted", "dashdot", "solid"]), label = f"{label}")
            
        plt.legend(loc="best" , fontsize="xx-large")
        plt.show()
        scores["seuil"] = seuil
    if give_results : return pd.DataFrame(scores ).set_index("seuil")

def knn_imputing_on_data(data  , save , nn = 5, m = 1500, verbose = False , AND = True) :
    """
    La fonction réalise un KNN_imputing sur les données
    
    Paramètre : 
    -----------
        data : pandas.core.frame.DataFrame
            DataFrame à utiliser par la fonction pour l'exécution du KNN-imputing
        save : Bool
            Defini si une sauvegarde des données doit être réalisée à la fin du KNN-imputing
        nn : int
            defini le nombre de plus proches voisins à prendre en considération dans l'algorithme de KNN-imputing
            la valeur par defaut est de 5
        m : int
            defini le nombre d'individus ou d'observations dans chaque regroupement  pour réaliser le KNN-imputing.
            l'objectif ici est de réduire le temps d'exécution de l'algorithme est découpant les données en plusieurs données de longueur m dans lesqiels ont réalise séparement le KNN-imputing
            La valeur par defaut est de 1500
        verbose : bool
            defini si l'algorithme doit écrire à chaque étape de son exécution afin que l'utilisateur puisse connaitre en temps réel l'etat d'execution de la tâche.
            La valeur par defaut est False
        AND : bool
            defini le booléen à utiliser pour la condition d'une boucle de KNN-imputing, la valeur par defaut est TRUE
        
    Return :  pandas.core.frame.DataFrame
    --------
        DataFrame modifié par le KNN-imputing
    
    """
    temps_i = time.time()
    n_classe = data.shape[0]//m
    data[data.isin([np.inf, -np.inf]).values] = np.nan  # Transformation des valeurs infinies en valeurs manquantes
    if AND : 
        condition = data.select_dtypes(include=[int,float]).isna().mean().mean() != 0  and "application_train_KNN.csv" not in os.listdir()
    else :
        condition = data.select_dtypes(include=[int,float]).isna().mean().mean() != 0  or "application_train_KNN.csv" not in os.listdir()
    while condition :
        knn_imputer = KNNImputer(n_neighbors=nn)  # Instantiation d'un imputer avec pour paramètre le nombre de 
        steps = np.linspace( start=0 , stop=data.shape[0] , num=n_classe  , dtype=int)

        for i in range(steps.size-2) :  
            data.select_dtypes(include=[int,float])[ steps[i]:steps[i+1] ] = knn_imputer.fit_transform(  data.select_dtypes( include=[ int , float ] )[ steps[i]:steps[i+1] ]  )
            if i % 15 == 0 and verbose :
                print( f"- Etape {i} / {n_classe} -   {round(100*i*m/data.shape[0] , 2)} % terminé  - cluster de {m} lignes - KNN = {nn}" )
        data.select_dtypes(include=[int,float])[ steps[-2]: ] = knn_imputer.fit_transform(  data.select_dtypes( include=[ int , float ] )[ steps[-2]: ]  )
        if verbose : print(f"- Etape {n_classe} / {n_classe} -   100.0 % terminé  - cluster de {m} lignes")
        for i in data.select_dtypes(int).columns :
            data[i] = data[i].apply(round , args= (0))  # Arrondi de tous les valeurs entières
        data["CNT_FAM_MEMBERS"] = data["CNT_FAM_MEMBERS"].apply(round , args= (0))
        
        data[data.isin([np.inf, -np.inf]).values] = np.nan  # Transformation des valeurs infinies en valeurs manquantes
        #data[(~np.isfinite(data.values)).values] = np.nan  # Transformation des valeurs infinies en valeurs manquantes
        save = True
        
        if AND : 
            condition = data.select_dtypes(include=[int,float]).isna().mean().mean() != 0  and "application_train_KNN.csv" not in os.listdir()
        else :
            condition = data.select_dtypes(include=[int,float]).isna().mean().mean() != 0  or "application_train_KNN.csv" not in os.listdir()
    print(f"KNN imputing terminé avec les {nn} premiers voisins\nDurée de l'execution {round(time.time()-temps_i,2)} secondes.")
    if save :
        print(f"Taille des données à la fin du traitement : {data.shape}")
        print("Sauvegarde des données en cours . . .")
        if "Unnamed: 0" in data.columns : 
            data = data.drop(["Unnamed: 0"], axis=1)
        data.to_csv("application_train_KNN.csv")  # Sauvegarde des données
        print("Sauvegarde des données terminée avec succès.")
    else :
        print("RAS par ici : Rien à signaler dans le coin")
    
    return data , save

def nouvelles_variables(data):
    """
    La fonction defini de nouvelles variables dans un Dataframe à partir de varaibles pré-existante dans les données.
    Les nouvelles variables calculées sont directement consevées dans les Dataframe fourni à la fonction
    
    Paramètres :
    ------------
        data : pandas.core.frame.DataFrame
            DataFrame à utiliser dans la fonction pour l'éaluation de nouvelles variables
    
    Return :
    --------
        data : pandas.core.frame.DataFrame
    
    """

    "CNT_CHILDREN","CNT_FAM_MEMBERS","AMT_INCOME_TOTAL",
    #1 - Nombre d'adultes dans la famille
    data["CNT_ADULT_TOTAL"] = data["CNT_FAM_MEMBERS"] - data["CNT_CHILDREN"]

    #2 -  Taux de remboursement du credit
    data["AMT_CREDIT_RATE_REPAYMENT"] = data["AMT_ANNUITY"]/data["AMT_CREDIT"]

    #3 -  Children proportion in familly
    data["CHILD_PROP_IN_FAMILLY"] = data["CNT_CHILDREN"]/data["CNT_FAM_MEMBERS"]

    #4 -  Parts des revenus relatifs aux enfants
    data["AMT_INCOME_RELATIF_TO_CHILDREN"] = data["AMT_INCOME_TOTAL"]*data["CNT_CHILDREN"]/data["CNT_FAM_MEMBERS"]
    
    #5 -  Parts des prêt à la consommation relatifs aux enfants
    data["AMT_GOOD_PRICE_RELATIF_TO_CHILDREN"] = data["AMT_GOODS_PRICE"]*data["CNT_CHILDREN"]/data["CNT_FAM_MEMBERS"]

    #6 -  revenu par membre de la famille
    data["AMT_INCOME_PER_FAMLY_MEMB"] = data["AMT_INCOME_TOTAL"]/data["CNT_FAM_MEMBERS"]

    #7 -  revenu par adulte de la famille
    data["AMT_INCOME_PER_ADULT"] = data["AMT_INCOME_TOTAL"]/data["CNT_ADULT_TOTAL"]

    #8 -  Revenu net de la famille
    data["AMT_INCOME_NET"] = data["AMT_INCOME_TOTAL"] - data["AMT_ANNUITY"]

    #9 - revenu net par membre de la famille
    data["AMT_INCOME_NET_PER_FAMLY_MEMB"] = (data["AMT_INCOME_TOTAL"] - data["AMT_ANNUITY"])/data["CNT_FAM_MEMBERS"]

    #10 - revenu net par adulte de la famille
    data["AMT_INCOME_NET_PER_ADULT"] = (data["AMT_INCOME_TOTAL"] - data["AMT_ANNUITY"])/data["CNT_ADULT_TOTAL"]

    #11 - Credit à la consommation par membre de la famille
    data["AMT_GOODS_PRICE_PER_FAMLY_MEMB"] = data["AMT_GOODS_PRICE"]/data["CNT_FAM_MEMBERS"]

    #12 - Credit à la consommation par adulte de la famille
    data["AMT_GOODS_PRICE_PER_ADULT"] = data["AMT_GOODS_PRICE"]/data["CNT_ADULT_TOTAL"]

    #13 - Credit non dédié à la consommation
    data["AMT_NOT_GOODS_PRICE"] = data["AMT_CREDIT"] - data["AMT_GOODS_PRICE"]

    #14 - Credit de non consommation par membre de la famille
    data["AMT_NOT_GOODS_PRICE_PER_FAMLY_MEMB"] = data["AMT_NOT_GOODS_PRICE"]/data["CNT_FAM_MEMBERS"]

    # Credit de non consommation par adulte de la famille
    data["AMT_NOT_GOODS_PRICE_PER_ADULT"] = data["AMT_NOT_GOODS_PRICE"]/data["CNT_ADULT_TOTAL"]

    #15 - Richesse réel de la famille
    data["AMT_EXACT_RICHNESS"] = data["AMT_INCOME_TOTAL"] - data["AMT_CREDIT"]

    # Richesse exacte par membre de la famille
    data["AMT_EXACT_RICHNESS_PER_FAMLY_MEMB"] = data["AMT_EXACT_RICHNESS"]/data["CNT_FAM_MEMBERS"]

    #16 - Richesse exacte par adulte de la famille
    data["AMT_EXACT_RICHNESS_PER_ADULT"] = data["AMT_EXACT_RICHNESS"]/data["CNT_ADULT_TOTAL"]

    #17 - pourcentage de salaire sur le credit
    data["AMT_INCOME_TOTAL_PER_CREDIT"] = data["AMT_INCOME_TOTAL"]/data["AMT_CREDIT"]

    #18 - Annuité par revenu
    data["AMT_ANNUITY_PER_INCOME"] = data["AMT_ANNUITY"]/data["AMT_INCOME_TOTAL"]

    #19 - Annuité par revenu net
    data["AMT_ANNUITY_PER_NET_INCOME"] = data["AMT_ANNUITY"]/(data["AMT_INCOME_TOTAL"] - data["AMT_ANNUITY"])

    #20 - Annuité par richesse réelle
    data["AMT_ANNUITY_PER_EXACT_RICHNESS"] = data["AMT_ANNUITY"]/data["AMT_EXACT_RICHNESS"]

    #21 - Age du client à la fin du prêt
    data["CLIENT_AGE_AT_END_REPAYMENT"] = data["DAYS_BIRTH"] + 1.*data["AMT_CREDIT"]/data["AMT_ANNUITY"]

    #22 - L'âge du client à son dernier emploi
    data["AGE_AT_LAST_JOB"] = data["DAYS_BIRTH"]  - data["DAYS_EMPLOYED"]

    #23 - Poucentage des jour d'enploi par rapport à l'âge
    data["DAYS_EMPLOYED_PERC"] = data["DAYS_EMPLOYED"]/data["DAYS_BIRTH"]
    
    #24 - Credit + credit à la conssomation
    data["AMT_CREDIT+GOODS_PRICE"] = data["AMT_CREDIT"] + data["AMT_GOODS_PRICE"]

    return data
